In [1]:
#%%capture
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on
root.gStyle.SetOptStat(0)

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOptions = [2,2,5,9,10,16,15]
keffs = [1.0,1.0,1.0,0.99,0.98,0.995,0.953]
centralities = ["0%-93%"]
Opt_names = ["no ML eID","ML eID", "conv partner","tighter match","conv pattern", "ML conv rej", "ML hit, ddphi"]

In [3]:
N_centr = 6
N_options = 6
N_pt = 1
colors0=[1,2,root.kGreen+2,4,root.kCyan,root.kMagenta,root.kOrange+4,root.kGray,root.kGray,root.kBlack,root.kYellow,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
colors=[2,4,4,root.kGreen+2,root.kOrange+4,root.kMagenta,root.kGray,root.kGray,root.kBlack,root.kCyan,root.kOrange,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
central_bins = [0,20,60,80,20,40,40,60,60,80,80,100]
pt_boarders = [0.4,0.7,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_path="input/data/"
file_names=["m_ee_DCA_531.root","m_ee_DCA_531.root","m_ee_DCA_532.root","m_ee_DCA_533.root","m_ee_DCA_534.root","m_ee_DCA_537.root"]#,"m_ee_DCA_535.root"],"m_ee_DCA_536.root"]
hist_names = [["inv_mass_stat_00","inv_mass_syst_00",f"inv_mass_ee_DCA_V{iOption}_FG12",f"inv_mass_ee_DCA_V{iOption}_BG12"] for iOption in iOptions]
N_type = len(hist_names[0])

In [4]:
hists_read = []

for iFile in range(len(file_names)):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hists_emc_read0.append(infile.Get(hist_names[iFile][itype]))
        try: hists_emc_read0[-1].SetDirectory(root.nullptr); hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"_{iFile}")
        except: 
            print(f"Error setting directory for {hist_names[iFile][itype]}{0} in file {file_names[iFile]}")
            break
        hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"{iFile}")
    hists_read.append(hists_emc_read0)

    infile.Close()


In [5]:
c0 = root.TCanvas(f"c0",f"c0",900,900)
legends =[]
hist0_stat = hists_read[0][0].Clone(); hist0_syst = hists_read[0][1].Clone()
hist1_stat = hists_read[4][0].Clone(); hist1_syst = hists_read[4][1].Clone()
hist2_stat = hists_read[5][0].Clone(); hist2_syst = hists_read[5][1].Clone()


Format_Hist_total(hist0_stat,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
hist0_stat.SetMarkerColorAlpha(colors0[0],(10-1)/10.); hist0_stat.SetLineColorAlpha  (colors0[0],(10-1)/10.)
hist0_syst.SetFillColorAlpha(colors0[0],0.4)

hist1_stat.SetMarkerColorAlpha(colors0[1],(10-1)/10.); hist1_stat.SetLineColorAlpha  (colors0[1],(10-1)/10.)
hist1_syst.SetFillColorAlpha(colors0[1],0.4)

hist2_stat.SetMarkerColorAlpha(colors0[2],(10-1)/10.); hist2_stat.SetLineColorAlpha  (colors0[2],(10-1)/10.)
hist2_syst.SetFillColorAlpha(colors0[2],0.4)

hist0_stat.Draw(""); hist0_syst.Draw("same e2")
hist2_stat.Draw("same"); hist2_syst.Draw("same e2")
hist1_stat.Draw("same"); hist1_syst.Draw("same e2")

root.gPad.SetLogy()
c0.Draw()


In [6]:
edges = array('d', [
    0.,0.025,0.05,0.075,0.1,0.15,0.25,0.35,0.45,0.65,0.7,0.75,0.8,0.9,0.95,1.0,1.05,
    1.15,1.3,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,
    3.55,3.65,3.75,3.85,4.5
])

nbins = len(edges) - 1

In [7]:
c1 = root.TCanvas(f"c1",f"c1",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = False
alphas = [0.5,0.3,0.3,0.3,0.3,0.3,0.3]
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        keff  = keffs[ifile]
        if if_divide: 
            if proj_BG.GetBinContent(ibin) >0:
                proj.SetBinContent(ibin,(proj.GetBinContent(ibin)/proj_BG.GetBinContent(ibin))/keff)
                proj.SetBinError(ibin,(proj.GetBinContent(ibin)**0.5/proj_BG.GetBinContent(ibin))/keff)
            else : 
                proj.SetBinContent(ibin,1)
        else:
            proj.SetBinContent(ibin,proj.GetBinContent(ibin) - keff*proj_BG.GetBinContent(ibin))
        if not if_divide: 
            proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
            proj_syst.SetBinContent(ibin,proj.GetBinContent(ibin))
        proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinError(ibin,abs(proj_BG.GetBinContent(ibin))**0.5)
        proj_syst.SetBinError(ibin,((proj_BG.GetBinContent(ibin)/100./proj.GetBinWidth(ibin))**2+0*(proj_BG.GetBinError(ibin)/proj.GetBinWidth(ibin))**2)**0.5)
        if if_divide: 
            proj.SetBinError(ibin,proj.GetBinError(ibin)/(proj_BG.GetBinContent(ibin)+1))
    
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj.Draw("")   
    legend = root.TLegend(0.55,0.75,0.89,0.89); legends.append(legend)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
        proj_syst.SetFillColorAlpha(colors0[ifile],alphas[ifile-ic])
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")
c1.Draw()

In [8]:
c1 = root.TCanvas(f"c1",f"c1",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = False
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        keff  = 0
        if if_divide: 
            if proj_BG.GetBinContent(ibin) >0:
                proj.SetBinContent(ibin,(proj.GetBinContent(ibin)/proj_BG.GetBinContent(ibin))/keff)
                proj.SetBinError(ibin,(proj.GetBinContent(ibin)**0.5/proj_BG.GetBinContent(ibin))/keff)
            else : 
                proj.SetBinContent(ibin,1)
        else:
            proj.SetBinContent(ibin,proj.GetBinContent(ibin) - keff*proj_BG.GetBinContent(ibin))
        if not if_divide: 
            proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
            proj_syst.SetBinContent(ibin,proj.GetBinContent(ibin))
        proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinError(ibin,abs(proj_BG.GetBinContent(ibin))**0.5)
        proj_syst.SetBinError(ibin,((proj_BG.GetBinContent(ibin)/100./proj.GetBinWidth(ibin))**2+0*(proj_BG.GetBinError(ibin)/proj.GetBinWidth(ibin))**2)**0.5)
        if if_divide: 
            proj.SetBinError(ibin,proj.GetBinError(ibin)/(proj_BG.GetBinContent(ibin)+1))
    
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj.Draw("")   
    legend = root.TLegend(0.55,0.75,0.89,0.89); legends.append(legend)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
        proj_syst.SetFillColorAlpha(colors0[ifile],alphas[ifile-ic])
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")
c1.Draw()

In [9]:
c1 = root.TCanvas(f"c2",f"c2",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = True
legend = root.TLegend(0.2,0.63,0.7,0.95); legends.append(legend)
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        if proj.GetBinContent(ibin) > 0:
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin)) / (keff * proj_BG.GetBinContent(ibin) +1e-48)-1)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48))
        proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)
    
    Format_Hist_total(proj,"m_{ee}, GeV","SG/BG, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj.Scale(0.2); proj_syst.Scale(0.2)
    if ifile == 0:  proj.Draw("")   
    proj.SetMaximum(3000)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst.SetFillColorAlpha(colors0[ifile],alphas[ifile-ic])
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")
c1.Draw()

In [10]:
c1 = root.TCanvas(f"c2",f"c2",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = True
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX2_{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_sys2_t{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX2_{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        error = 0
        if proj.GetBinContent(ibin) > 0:
            error = proj.GetBinError(ibin) / (proj.GetBinContent(ibin) - keff * proj_BG.GetBinContent(ibin)) 
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin) - keff * proj_BG.GetBinContent(ibin)) / proj.GetBinContent(ibin)**0.5 )
            error *= abs(proj.GetBinContent(ibin)**0.5)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, error)
        #proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48)**0.5)
        #proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)
    
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj.Draw("")   
    legend = root.TLegend(0.55,0.75,0.89,0.89); legends.append(legend)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst.SetFillColorAlpha(colors0[ifile],alphas[ifile-ic])
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")
c1.Draw()

In [11]:
c1 = root.TCanvas(f"c2",f"c2",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = True
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        if proj.GetBinContent(ibin) > 0:
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin)) / (keff * proj_BG.GetBinContent(ibin) +1e-48)-1)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48))
        proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)

    for ibin in range(1, 1 + proj_syst.GetNbinsX()):
        proj_syst.SetBinContent(ibin, proj_syst.GetBinContent(ibin)/proj_syst.GetBinError(ibin))
        proj_syst.SetBinError(ibin, 1)
    
    Format_Hist_total(proj_syst,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj_syst.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj_syst.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj_syst.Scale(0.2)
    if ifile == 0:  proj_syst.Draw("")   
    legend = root.TLegend(0.55,0.75,0.89,0.89); legends.append(legend)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj_syst,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        if ifile == 0 and ic !=0: proj_syst.Draw("")
        else: proj_syst.Draw("same")
c1.Draw()

In [12]:
c1 = root.TCanvas(f"c1",f"c1",900,900)
legends =[]
hist0_new_stat = hist0_stat.Rebin(1, hist0_stat.GetName()+"hist0_stat_rebinned"); hist0_new_syst = hist0_syst.Rebin(1, hist0_syst.GetName()+"hist0_syst_rebinned")
hist1_new_stat = hist1_stat.Rebin(1, hist1_stat.GetName()+"hist1_stat_rebinned"); hist1_new_syst = hist1_syst.Rebin(1, hist1_syst.GetName()+"hist1_syst_rebinned")
hist2_new_stat = hist2_stat.Rebin(1, hist2_stat.GetName()+"hist2_stat_rebinned"); hist2_new_syst = hist2_syst.Rebin(1, hist2_syst.GetName()+"hist2_syst_rebinned")  



for ibin in range(1,hist0_new_stat.GetNbinsX()+1):#dividing hist1 adh hist2  by hist0
    val0 = hist0_new_stat.GetBinContent(ibin)
    val1 = hist1_new_stat.GetBinContent(ibin)
    val2 = hist2_new_stat.GetBinContent(ibin)
    if val0>0:
        hist1_new_stat.SetBinContent(ibin, val1/val0)
        hist2_new_stat.SetBinContent(ibin, val2/val0)
        err1 = hist1_new_stat.GetBinError(ibin)
        err2 = hist2_new_stat.GetBinError(ibin)
        hist1_new_stat.SetBinError(ibin, err1/val0)
        hist2_new_stat.SetBinError(ibin, err2/val0)
        hist0_new_syst.SetBinContent(ibin, 1)
        hist0_new_syst.SetBinError(ibin, hist0_new_syst.GetBinError(ibin)/val0)
    else:
        hist1_new_stat.SetBinContent(ibin, 0)
        hist2_new_stat.SetBinContent(ibin, 0)
        hist1_new_stat.SetBinError(ibin, 0)
        hist2_new_stat.SetBinError(ibin, 0)

Format_Hist_total(hist1_new_stat,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
hist0_new_stat.SetMarkerColorAlpha(colors0[0],(10-1)/10.); hist0_new_stat.SetLineColorAlpha  (colors0[0],(10-1)/10.)
hist0_new_syst.SetFillColorAlpha(colors0[0],0.3)

hist1_new_stat.SetMarkerColorAlpha(colors0[1],(10-1)/10.); hist1_new_stat.SetLineColorAlpha  (colors0[1],(10-1)/10.)
hist1_new_syst.SetFillColorAlpha(colors0[1],0.2)

hist2_new_stat.SetMarkerColorAlpha(colors0[2],(10-1)/10.); hist2_new_stat.SetLineColorAlpha  (colors0[2],(10-1)/10.)
hist2_new_syst.SetFillColorAlpha(colors0[2],0.2)

hist1_new_syst.Divide(hist0_new_syst); hist2_new_syst.Divide(hist0_new_syst)
#root.gPad.SetLogy(1)
hist1_new_stat.SetMaximum(2); hist1_new_stat.SetMinimum(0.1)
hist1_new_stat.Draw(""); #hist1_new_syst.Draw("same e2")
hist2_new_stat.Draw("same"); #hist2_new_syst.Draw("same e2")
hist0_new_syst.Draw("same e2")
#drawing dashed line at y=1
line = root.TLine(hist1_new_stat.GetXaxis().GetXmin(), 1, hist1_new_stat.GetXaxis().GetXmax(), 1)
line.SetLineStyle(2); line.SetLineWidth(4); line.SetLineColor(root.kBlack)
line.Draw("same")

c1.Draw()


In [13]:
c1 = root.TCanvas(f"c2",f"c2",1000,1000)
legends =[]
if_divide = True
legend = root.TLegend(0.2,0.58,0.7,0.9); legends.append(legend)
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX1{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_1syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_po1rjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        if proj.GetBinContent(ibin) > 0:
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin) - keff * proj_BG.GetBinContent(ibin) ) / (keff * proj_BG.GetBinContent(ibin) + proj.GetBinContent(ibin) +  1e-48)**0.5)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48))
        proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)
    
    Format_Hist_total(proj,"m_{e^{+}e^{-}} [GeV]","Signal/#sqrt{SG+BG}",0.17,0.13,0.01,0.01,0.06,0.05,21,1.5,4,3,4,0.85,1.34)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.); proj_syst.SetMarkerColorAlpha(0,0)
    root.gPad.SetLogy(); 
    if ifile == 0:  
        rand0 = root.gRandom.Rndm() * 5.0/20.0 + 0.1
        for ibin in range(1, 1 + proj.GetNbinsX()): proj.SetBinContent(ibin, proj.GetBinContent(ibin)*rand0)
        for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinContent(ibin, proj_syst.GetBinContent(ibin)*rand0)
    if ifile == 0:  proj.Draw("")   
    proj.SetMaximum(3000)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst.SetFillColorAlpha(colors0[ifile],0.1)
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")

legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[0]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
Format_Legend(legends[-1],0.05,12)
legends[-1].Draw()
c1.Draw()
#c1.SaveAs("output/mass/SG_BG_comparison.pdf")

In [14]:
c1 = root.TCanvas(f"c2",f"c2",1000,1000)
legends =[]
if_divide = True
legend = root.TLegend(0.2,0.58,0.7,0.9); legends.append(legend)
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        if proj.GetBinContent(ibin) > 0:
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin)) / (keff * proj_BG.GetBinContent(ibin) +1e-48)-1)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48))
        proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)
    
    Format_Hist_total(proj,"m_{e^{+}e^{-}} [GeV]","Signal/Background",0.17,0.13,0.01,0.01,0.06,0.05,21,1.5,4,3,4,0.85,1.34)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.); proj_syst.SetMarkerColorAlpha(0,0)
    root.gPad.SetLogy(); 
    if ifile == 0:  
        rand0 = root.gRandom.Rndm() * 5.0/20.0 + 0.1
        for ibin in range(1, 1 + proj.GetNbinsX()): proj.SetBinContent(ibin, proj.GetBinContent(ibin)*rand0)
        for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinContent(ibin, proj_syst.GetBinContent(ibin)*rand0)
    if ifile == 0:  proj.Draw("")   
    proj.SetMaximum(3000)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst.SetFillColorAlpha(colors0[ifile],0.1)
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")

legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[0]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
Format_Legend(legends[-1],0.05,12)
legends[-1].Draw()
c1.Draw()
#c1.SaveAs("output/mass/SG_BG_comparison.pdf")

In [15]:
infile = root.TFile.Open("input/Star.root", "READ")

d = infile.Get("Table 1")                  # TDirectoryFile
hists_star = d.Get("Graph1D_y1")           # TGraphAsymmErrors

# Optional safety check
if not hists_star:
    raise RuntimeError("Graph not found")

hists_star.SetName("STAR_AuAu")             # optional
hists_star.SetTitle("STAR Au+Au 0–80%")

infile.Close()

In [16]:

# ---- extract x values once ----
n = hists_star.GetN()
x = hists_star.GetX()
#  ---- set x errors from neighbor spacing ----
for i in range(n):
    if i == 0:
        dx = 0.5 * (x[1] - x[0])
    elif i == n - 1:
        dx = 0.5 * (x[n-1] - x[n-2])
    else:
        dx = 0.5 * (x[i+1] - x[i-1])

    hists_star.SetPointEXlow(i,  dx/2)
    hists_star.SetPointEXhigh(i, dx/2)


In [17]:
accepance_eff_low_mass = root.TF1("accepance_eff_low_mass","pol1",0,0.7)
accepance_eff_low_mass.SetParameters(1.28898e-05,5.861e-06)
accepance_eff_high_mass = root.TF1("accepance_eff_high_mass","pol1",0.7,5.0 )
accepance_eff_high_mass.SetParameters(-5.34624e-05,0.000101395)

for ibin in range(1,hists_read[len(file_names)-1][0].GetNbinsX()+1):#dividing hist1 adh hist2  by hist0
    mass = hists_read[len(file_names)-1][0].GetBinCenter(ibin)
    if mass < 0.7:
        eff = accepance_eff_low_mass.Eval(mass)
    elif mass < 2.9:
        eff = accepance_eff_high_mass.Eval(mass)
    else: eff = accepance_eff_high_mass.Eval(2.5)
    if eff > 0:
        hists_read[len(file_names)-1][0].SetBinContent(ibin, hists_read[len(file_names)-1][0].GetBinContent(ibin)/eff)
        hists_read[len(file_names)-1][0].SetBinError(ibin, hists_read[len(file_names)-1][0].GetBinError(ibin)/eff)
        hists_read[len(file_names)-1][1].SetBinContent(ibin, hists_read[len(file_names)-1][1].GetBinContent(ibin)/eff)
        hists_read[len(file_names)-1][1].SetBinError(ibin, hists_read[len(file_names)-1][1].GetBinError(ibin)/eff)
    

In [18]:
dnch_phenix = 550
dnch_star  = 180
iOpt = len(file_names)-1
hists_star.Scale(1./dnch_star)
hists_read[iOpt][0].Scale(1./dnch_phenix)
hists_read[iOpt][1].Scale(1./dnch_phenix)

In [19]:
# green markers: Centrality 20-40% (digitized)
eff_mass_raw = [
    0.0514, 0.1210, 0.1222, 0.1763, 0.2763,
    0.4019, 0.5120, 0.6615, 0.7715, 0.8911,
    1.0120, 1.0622, 1.1471, 1.3517, 1.6268,
    1.8768, 2.1017, 2.3505, 2.6519, 3.9007
]

eff_raw = [
    0.07919, 0.08142, 0.07363, 0.07728, 0.06358,
    0.06854, 0.06654, 0.07447, 0.07810, 0.07883,
    0.07477, 0.07875, 0.07919, 0.08348, 0.09110,
    0.08899, 0.09044, 0.09443, 0.09843, 0.09407
]
# bin edges from the table (variable width bins)
mass_edges = [
    0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10,
    0.15, 0.25, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05,
    1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 2.95, 3.00,
    3.05, 3.15, 3.25, 3.35, 3.75, 4.50, 5.00
]

# bin centers
mass_centers = [
    0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095,
    0.125, 0.200, 0.350, 0.500, 0.600, 0.700, 0.800, 0.900,
    1.000, 1.150, 1.375, 1.625, 1.875, 2.125, 2.375, 2.625,
    2.850, 2.975, 3.025, 3.100, 3.200, 3.300, 3.550, 4.125,
    4.750
]

# dN/dm_ee in PHENIX acceptance (GeV/c^2)^-1 from the table
yield_acc = [
    0.00478, 0.01769, 0.02320, 0.02107, 0.01643, 0.01288, 0.00949, 0.00653,
    0.00198, 0.00087, 0.000348, 0.00027, 0.00011, 0.000202, 0.000273, 0.000127,
    0.000124, 3.4e-05, 3.9e-05, 1.02e-05, 9.2e-06, 5.2e-06, 1.26e-06, 1.93e-06,
    4.0e-06, 6.8e-06, 9.0e-06, 1.38e-05, 5.8e-06, 1.38e-06, 3.60e-07, 9.3e-08,
    9.2e-08
]

# statistical uncertainties
stat_acc = [
    1.0e-04, 1.9e-04, 2.3e-04, 2.4e-04, 2.7e-04, 2.8e-04, 3.1e-04, 3.2e-04,
    1.3e-04, 1.0e-04, 8.7e-05, 1.0e-04, 7.0e-05, 7.0e-05, 1.3e-04, 3.3e-05,
    6.0e-05, 1.8e-05, 1.0e-05, 6.0e-06, 4.0e-06, 2.4e-06, 1.4e-06, 9.0e-07,
    8.0e-07, 1.4e-06, 1.3e-06, 9.0e-07, 7.0e-07, 6.0e-07, 1.9e-07, 7.0e-08,
    4.0e-08
]

# systematic uncertainties
# NOTE: a few bins in the screenshot are asymmetric; here I stored the *average* of up/down as a single sigma.
sys_acc = [
    5.0e-04, 1.7e-03, 2.3e-03, 2.05e-03, 1.65e-03, 1.35e-03, 9.0e-04, 6.0e-04,
    2.0e-04, 9.0e-05, 4.8e-05, 5.0e-05, 5.0e-05, 3.3e-05, 3.4e-05, 1.9e-05,
    2.25e-05, 7.5e-06, 4.5e-06, 1.4e-06, 9.5e-07, 5.0e-07, 1.35e-07, 1.9e-07,
    4.0e-07, 7.0e-07, 9.0e-07, 1.3e-06, 6.0e-07, 1.3e-07, 3.5e-08, 9.0e-09,
    9.0e-09
]
# efficiency evaluated at each mass_centers (linear interp; clamped outside [0.0514, 3.9007])
eff_at_centers = [
    0.07919, 0.07919, 0.07919, 0.079305345, 0.079625747, 0.079946149, 0.080266552, 0.080586954,
    0.073818909, 0.0740331, 0.066490446, 0.066757984, 0.071207826, 0.0757405, 0.078273955, 0.078531125,
    0.075172978, 0.079250806, 0.084125387, 0.091050142, 0.089005192, 0.090813662, 0.094755149, 0.098072999,
    0.097738363, 0.097301944, 0.097127377, 0.096865525, 0.09651639, 0.096167255, 0.095294417, 0.09407,
    0.09407
]

# corrected yield and uncertainties: (table)/eff_at_centers
yield_corr = [
    0.060361157, 0.22338679, 0.29296628, 0.26568197, 0.20634029, 0.16110845, 0.11823107, 0.081030485,
    0.026822396, 0.0117515, 0.0052338347, 0.0040444601, 0.001544774, 0.0026670011, 0.0034877502, 0.0016171932,
    0.0016495289, 0.00042901772, 0.00046359371, 0.00011202618, 0.00010336476, 5.7260107e-05, 1.329743e-05,
    1.9679219e-05, 4.0925588e-05, 6.9885551e-05, 9.2661825e-05, 0.00014246555, 6.009342e-05, 1.4349999e-05,
    3.7777659e-06, 9.8862549e-07, 9.7799511e-07
]

stat_corr = [
    0.0012627857, 0.0023992928, 0.0029044071, 0.0030262777, 0.003390863, 0.0035023575, 0.0038621318, 0.0039708661,
    0.0017610664, 0.0013507472, 0.0013084587, 0.0014979482, 0.00098303801, 0.00092420832, 0.0016608334, 0.00042021555,
    0.00079815915, 0.00022712703, 0.00011887018, 6.5897756e-05, 4.4941198e-05, 2.6427742e-05, 1.4774923e-05,
    9.1768377e-06, 8.1851176e-06, 1.4388202e-05, 1.3384486e-05, 9.2912313e-06, 7.2526542e-06, 6.2391299e-06,
    1.9938209e-06, 7.4412671e-07, 4.2521527e-07
]

sys_corr = [
    0.0063139285, 0.021467357, 0.029044071, 0.025849456, 0.020721941, 0.016886367, 0.011212641, 0.0074453738,
    0.0027093329, 0.0012156724, 0.00072190823, 0.00074897409, 0.00070217001, 0.00043569821, 0.00043437182,
    0.00024194229, 0.00029930968, 9.4636261e-05, 5.3491581e-05, 1.5376143e-05, 1.0673535e-05, 5.5057795e-06,
    1.4247247e-06, 1.9373324e-06, 4.0925588e-06, 7.1941009e-06, 9.2661825e-06, 1.3420667e-05, 6.2165607e-06,
    1.3518115e-06, 3.672828e-07, 9.5673435e-08, 9.5673435e-08
]


In [20]:
sim_file_names = ['ccbar_soft_v06.root','bbbar_v03.root','thermal_300_v04.root','phi_25M_v04.root']
hists_sim = []
hist_select_3D_names = ["inv_mass_dca_gen_5","inv_mass_dca_gen_6","inv_mass_dca_gen_7","inv_mass_dca_gen_8","inv_mass_dca_gen_9"]
sim_file_path = "../sim/output/fall25/dca/"
for iFile in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(len(hist_select_3D_names)):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)
    infile.Close()

In [ ]:
for iFile in range(len(sim_file_names)):
    for ihist in range(1,len(hist_select_3D_names)): 
        hists_sim[iFile][0].Add(hists_sim[iFile][ihist])


hists_sim_porj_ccbar = hists_sim[0][0].ProjectionY(hists_sim[0][0].GetName()+f"_projY",1,80,1,25)
hists_sim_porj_bbbar = hists_sim[1][0].ProjectionY(hists_sim[1][0].GetName()+f"_projY",1,80,1,25)
hists_sim_porj_thermal = hists_sim[2][0].ProjectionY(hists_sim[2][0].GetName()+f"_projY",1,80,1,25)
hists_sim_porj_phi = hists_sim[3][0].ProjectionY(hists_sim[3][0].GetName()+f"_projY",1,80,1,25)
br =0.01
abs_norm_cc_new = 5000/3.5e7/10*br * 2
abs_norm_bb = 0.001102 * 10000 / 2104191 / 42 / 10 * 2
thermal_norm = 1.5e-7/0.156/3
hists_sim_porj_ccbar.Scale(1./hists_sim[0][0].GetEntries()*abs_norm_cc_new*10/hists_sim_porj_ccbar.GetBinWidth(1)/dnch_phenix)
hists_sim_porj_bbbar.Scale(1./hists_sim[1][0].GetEntries()*abs_norm_bb*10/hists_sim_porj_bbbar.GetBinWidth(1)/dnch_phenix)
hists_sim_porj_thermal.Scale(1./hists_sim[2][0].GetEntries()*thermal_norm*10/hists_sim_porj_thermal.GetBinWidth(1)/dnch_phenix)
hists_sim_porj_phi.Scale(1./hists_sim[3][0].GetEntries()*10/hists_sim_porj_phi.GetBinWidth(1)/dnch_phenix)

In [22]:
thermal_spectrum_change = root.TF1("thermal_spectrum_change","[0]*(x-[1])*(x-[1])+[2]",0,10)
thermal_spectrum_change.SetParameters(0.55,2.7,0.4)
for ibin in range(1,hists_sim_porj_thermal.GetNbinsX()+1):
    mass = hists_sim_porj_thermal.GetBinCenter(ibin)
    #mass = max(mass,0.7)
    hists_sim_porj_thermal.SetBinContent(ibin, hists_sim_porj_thermal.GetBinContent(ibin)*thermal_spectrum_change.Eval(mass))

In [23]:
infile = root.TFile.Open("../../pp_ee_analysis/sim/output/mass/prompt_v01.root", "read")
acceptance_hist = infile.Get("phenix_2arms_acceptance")
acceptance_proj = acceptance_hist.SetDirectory(root.nullptr)
infile.Close()

In [24]:
full_acceptance = acceptance_hist.ProjectionX(acceptance_hist.GetName()+f"_full",1,10,1,1)
phenix_acceptance = acceptance_hist.ProjectionX(acceptance_hist.GetName()+f"_phenix",1,10,2,2)
phenix_acceptance.Divide(full_acceptance)
c2 = root.TCanvas(f"c2",f"c2",400,400)
Format_Hist_total(phenix_acceptance,"m_{ee}, GeV","Acceptance",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
phenix_acceptance.Draw("")
phenix_acceptance.Rebin(5)
phenix_acceptance.Scale(0.2)
phenix_acceptance.Smooth(2)
c2.Draw()

In [25]:
hbd_paper_edges = array('d', mass_edges)
hbd_paper = root.TH1D("hbd_paper", "Corrected; m_{ee} (GeV/c^{2}); (dN/dm) / #epsilon", len(mass_centers), hbd_paper_edges)

for i, (y, e, eff) in enumerate(zip(yield_acc, stat_acc,eff_at_centers), start=1):
    corr = phenix_acceptance.GetBinContent(phenix_acceptance.FindBin(hbd_paper.GetBinCenter(i)))+1e-14
    hbd_paper.SetBinContent(i, y/dnch_phenix/corr)
    hbd_paper.SetBinError(i, e/dnch_phenix/corr)   # stat error as TH1 error

In [26]:
import numpy as np

# --- Digitized c\bar{c} -> ee from right panel (sqrt(sNN)=27 GeV) ---
# M_ee in GeV/c^2 ; dN/dM in (c^2/GeV)
mee_ccbar_star = np.array([
 0.08139, 0.22093, 0.35465, 0.51744, 0.66860, 0.79651, 0.98255, 1.16279, 1.33139, 1.53488, 1.71511, 1.84883, 1.96511, 2.09883, 2.23837, 2.38953, 2.59302
], dtype=float)

dndm = np.array([
     0.00005359282227808751, 0.00014309339789044228, 0.000263480985725280, 0.0003008750442442845, 0.000256579469002744, 0.0002020577461377244, 0.0001321409619342679, 0.00007980264593818095, 0.00005218903308949766, 0.000028343244795582354, 0.000016668711014599265, 0.00001090094069530079, 0.0000075176255805536485, 0.000004916348393784993, 0.000003215174960508243, 0.0000019939387595524385, 0.000001112012092778792
], dtype=float)


scale_star = dnch_star if dnch_star else 1.0
y = dndm / scale_star

# ---- Variable bin edges from midpoints ----
edges_ccbar_star = np.zeros(len(mee_ccbar_star) + 1, dtype=float)
edges_ccbar_star[1:-1] = 0.5 * (mee_ccbar_star[:-1] + mee_ccbar_star[1:])
edges_ccbar_star[0]    = mee_ccbar_star[0]  - 0.5 * (mee_ccbar_star[1] - mee_ccbar_star[0])
edges_ccbar_star[-1]   = mee_ccbar_star[-1] + 0.5 * (mee_ccbar_star[-1] - mee_ccbar_star[-2])

# ---- Histogram: contents are dN/dM (not counts), so Fill() is not used ----
hist_ccbar_star = root.TH1D(
    "h_ccbar_27GeV",
    "c#bar{c} #rightarrow e^{+}e^{-} (27 GeV);M_{ee} (GeV/c^{2});dN/dM (c^{2}/GeV)",
    len(mee_ccbar_star), edges_ccbar_star
)

for ibin, val in enumerate(y, start=1):
    hist_ccbar_star.SetBinContent(ibin, float(val))
    hist_ccbar_star.SetBinError(ibin, 0.0)

# ---- Draw ----


In [27]:
canvases = []
legends =[]
for ipar in [0]:#[0,1,2,3,4,5]:
    c0 = root.TCanvas(f"c0_{ipar}",f"c0_{ipar}",1000,1000)
    c0.Divide(1,1)
    canvases.append(c0)
    iOpt = len(file_names)-1
    rebin = 1

    centr_length = [1,0.2/0.93,0.2/0.93,0.2/0.93,0.2/0.93,0.13/0.93]
    for icentr in range(ipar,ipar+1):
        legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[icentr]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()
        c0.cd(1)
        root.gPad.SetLogy()
        Format_Hist_total(hists_read[iOpt][0],"m_{e^{+}e^{-}} [GeV]","dN/dm_{e^{+}e^{-}}#divide dN_{ch}/d#eta|_{#eta=0} [GeV^{-1}]",0.175,0.13,0.01,0.01,0.06,0.05,21,1.5,1,3,1,0.99,1.35)
        hists_read[iOpt][0].Draw("")
        hists_read[iOpt][1].SetFillColorAlpha(1,0.4)
        hists_read[iOpt][1].Draw("same e2")
        legends.append(root.TLegend(0.25,0.83,0.99,0.89,"p_{T}^{e }>0.3 GeV, |y^{e}|<0.35"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()

        legends.append(root.TLegend(0.25,0.63,0.8,0.92))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].AddEntry(0,"","")
        legends[-1].AddEntry(hists_read[iOpt][0],"PHENIX","pl")
        legends[-1].AddEntry(hists_star,"#splitline{STAR #sqrt{s_{NN}} = 27 GeV}{0-80% Au+Au}","pl")
        #legends[-1].AddEntry(hists_star,"STAR #sqrt{s_{NN}} = 27 GeV 0-80% Au+Au","pl")
        legends[-1].Draw()
        


        #sum_sim_proj.Draw("same L")
        hists_read[iOpt][0].SetMaximum(2e-1)
        hists_read[iOpt][0].SetMinimum(5e-9)

        Format_Graph(hists_star,Msize=2,Mstyle=29,Lcolor=2,Mcolor=2,Lwidth=2)
        hists_star.Draw("P same")
        Format_Hist_total(hbd_paper,"m_{e^{+}e^{-}} [GeV]","(1/N_{ch}) dN/dm_{e^{+}e^{-}} [GeV^{-1}]",0.175,0.13,0.01,0.01,0.06,0.05,21,1.5,4,3,4,0.99,1.35)
        #hbd_paper.Scale(5)
        hbd_paper.Draw("same p")
        hists_sim_porj_ccbar.SetLineColor(root.kBlue+1)
        hists_sim_porj_ccbar.SetLineWidth(1)
        hists_sim_porj_bbbar.SetLineColor(root.kRed+1)
        hists_sim_porj_bbbar.SetLineWidth(1)
        hists_sim_porj_thermal.SetLineColor(root.kGreen+1)
        hists_sim_porj_thermal.SetLineWidth(1)
        hists_sim_porj_ccbar.Draw("same hist")
        hists_sim_porj_bbbar.Draw("same hist")
        hists_sim_porj_thermal.Draw("same hist")
        hists_sim_proj_ccbar_bbar = hists_sim_porj_ccbar.Clone("hists_sim_proj_ccbar_bbar")
        hists_sim_proj_ccbar_bbar.Add(hists_sim_porj_bbbar)
        hists_sim_proj_ccbar_bbar.Add(hists_sim_porj_thermal)
        hists_sim_proj_ccbar_bbar.SetLineColor(root.kMagenta+2)
        hists_sim_proj_ccbar_bbar.SetLineWidth(3)
        hists_sim_proj_ccbar_bbar.Draw("same hist")
        hists_sim_porj_phi.SetLineColor(root.kOrange+7)
        hists_sim_porj_phi.SetLineWidth(2)
        hists_sim_porj_phi.Draw("same hist")

        hist_ccbar_star.SetLineColor(root.kCyan+2)
        hist_ccbar_star.SetLineWidth(2)
        hist_ccbar_star.Draw("same hist")
        #hbd_paper.SetMaximum(2e-1)

    #if True: root.PHENIXTools.DrawPreliminary(0.6, 0.6, 0.3)


    # Draw the logo on the canvas with specified coordinates and size
    #logo.DrawImage(0.1, 0.8, 0.3, 0.9)  # Adjust the coordinates and size as needed

    c0.Draw()
    #c0.Print(f"output/mass/mee_star.pdf")